## CNN

In [ ]:
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np

2025-08-20 14:40:09.951909: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-20 14:40:10.038937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755726010.061388   25203 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755726010.068033   25203 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-20 14:40:10.180832: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Preprocessing the Images

In [ ]:
# Convert lists to NumPy arrays
img_scale = (64, 64) # size to scale the images to. could test larger but kernel might crash again.

X_images = []
y_disaster = []
y_damage = []

disaster_list = list(data.keys())
disaster_to_idx = {d: i for i, d in enumerate(disaster_list)} 

for disaster, contents in data.items():
    imgs = contents["images"]   # list of numpy arrays (varied shapes)
    labels = contents["labels"] # damage levels
    d_idx = disaster_to_idx[disaster]
    
    for img, damage in zip(imgs, labels):
        # Resize images
        img_resized = tf.image.resize(img, img_scale).numpy()
        
        # Normalize
        img_resized = img_resized / 255.0
        
        X_images.append(img_resized)
        y_damage.append(damage)
        y_disaster.append(d_idx)

X_images = np.array(X_images, dtype=np.float32)
y_disaster = np.array(y_disaster)
y_damage   = np.array(y_damage)

I0000 00:00:1755726014.445332   25203 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5592 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [ ]:
num_disasters = len(disaster_list)
num_damage_levels = len(set(y_damage))

y_disaster = to_categorical(y_disaster, num_classes=num_disasters)
y_damage   = to_categorical(y_damage,   num_classes=num_damage_levels)

In [ ]:
from sklearn.model_selection import train_test_split

# Make training and testing sets from preprocessed data
# X is image data, two y's because multi output (disaster type and damage level)
X_train, X_test, y_disaster_train, y_disaster_test, y_damage_train, y_damage_test = train_test_split(
    X_images, y_disaster, y_damage, 
    test_size=0.2, random_state=42, stratify=y_disaster
)

### Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Number of classes from preprocessing step (one hot encoded)
# could be hard coded I guess but this makes it more adaptable?
num_disasters = y_disaster.shape[1]
num_damage_levels = y_damage.shape[1]

# Input layer
input_layer = layers.Input(shape=(64, 64, 3))

# Convolutional layers
x = layers.Conv2D(32, 3, activation='relu')(input_layer)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.GlobalMaxPooling2D()(x)

# Fully connected layers
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.3)(x)

# Two outputs (disaster type and damage level)
disaster_output = layers.Dense(num_disasters, activation='softmax', name='disaster_type')(x)
damage_output   = layers.Dense(num_damage_levels, activation='softmax', name='damage_level')(x)

# Build the model
model = models.Model(inputs=input_layer, outputs=[disaster_output, damage_output])

# Using categorical crossentropy loss for both tasks
# TODO: consider other loss options?
model.compile(
    optimizer='adam',
    loss={
        'disaster_type': 'categorical_crossentropy',
        'damage_level': 'categorical_crossentropy'
    },
    metrics={
        'disaster_type': 'accuracy',
        'damage_level': 'accuracy'
    }
)

# Print model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 62, 62,    │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 31, 31,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 29, 29,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 14, 14,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 12, 12,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv2d_2[0][0]    │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │      4,128 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disaster_type       │ (None, 3)         │         99 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ damage_level        │ (None, 4)         │        132 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 97,607 (381.28 KB)

 Trainable params: 97,607 (381.28 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train,
    {'disaster_type': y_disaster_train, 'damage_level': y_damage_train},
    validation_data=(X_test, {'disaster_type': y_disaster_test, 'damage_level': y_damage_test}),
    epochs=20,
    batch_size=8
)

Epoch 1/20


I0000 00:00:1755726068.499984   25279 service.cc:148] XLA service 0x76a3fc0027e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755726068.500014   25279 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2025-08-20 14:41:08.546872: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755726068.706098   25279 cuda_dnn.cc:529] Loaded cuDNN version 90101


  51/2654 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - damage_level_accuracy: 0.5668 - damage_level_loss: 1.1812 - disaster_type_accuracy: 0.4308 - disaster_type_loss: 1.1048 - loss: 2.2859 

I0000 00:00:1755726070.666717   25279 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2654/2654 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - damage_level_accuracy: 0.6409 - damage_level_loss: 0.9281 - disaster_type_accuracy: 0.6717 - disaster_type_loss: 0.7319 - loss: 1.6601 - val_damage_level_accuracy: 0.6946 - val_damage_level_loss: 0.7852 - val_disaster_type_accuracy: 0.8270 - val_disaster_type_loss: 0.4337 - val_loss: 1.2193
Epoch 2/20
2654/2654 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - damage_level_accuracy: 0.6838 - damage_level_loss: 0.7811 - disaster_type_accuracy: 0.8477 - disaster_type_loss: 0.4015 - loss: 1.1826 - val_damage_level_accuracy: 0.7123 - val_damage_level_loss: 0.7299 - val_disaster_type_accuracy: 0.8630 - val_disaster_type_loss: 0.3754 - val_loss: 1.1057
Epoch 3/20
2654/2654 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - damage_level_accuracy: 0.7054 - damage_level_loss: 0.7269 - disaster_type_accuracy: 0.8838 - disaster_type_loss: 0.3119 - loss: 1.0389 - val_damage_level_accuracy: 0.7200 - val_damage_level_loss: 0.6749 - val_disaster_type_accuracy: 0.9124 - val_disaster_t

In [ ]:
results = model.evaluate(
    X_test, 
    {'disaster_type': y_disaster_test, 'damage_level': y_damage_test}, 
    batch_size=32
)
print(results)


166/166 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - damage_level_accuracy: 0.7846 - damage_level_loss: 0.5546 - disaster_type_accuracy: 0.9546 - disaster_type_loss: 0.1346 - loss: 0.6893
[0.6892766356468201, 0.13464434444904327, 0.5545526146888733, 0.7846240997314453, 0.9545882940292358]
